### 1. Get the model and familiarize yourself with the iSB619

#### 1.1 Download the iSB619 model from BIGG and read it in with libsbml

hint: 

http://bigg.ucsd.edu/models/iSB619

In [2]:
import tellurium as te;
from libsbml import *;
import ast
import cobra;

reader = SBMLReader()
document = reader.readSBMLFromFile('iSB619.xml')

model = document.getModel()

cobra_model = cobra.io.read_sbml_model('iSB619.xml')
cobra.io.save_json_model(cobra_model, "iSB619_test.json")

#cobra.io.save_json_model(model, "original_model.json")

getLibSBMLVersion()

/opt/anaconda2/lib/python2.7/site-packages/tellurium/notebooks/__init__.py:16: UserWarning: Notebook tools are not imported, due to missing dependencies.
  warnings.warn("Notebook tools are not imported, due to missing dependencies.")


51300

#### 1.2 How many metabolites, reactions and compartments does the model have? 

In [3]:
model = model.getModel()

print 'species: ' + str(model.getNumSpecies())
print 'reactions: ' + str(model.getNumReactions())
print 'compartments: ' + str(model.getNumCompartments())

species: 655
reactions: 743
compartments: 2


#### 1.3 What are the compartments of the model?

In [109]:
def extract_sbml_by_group(sbml_document, group_id, destination):    
    reduced_document = SBMLDocument(sbml_document)
    rmodel = reduced_document.getModel()
    rgroupsplugin = rmodel.getPlugin("groups")
    rgroupslist = rgroupsplugin.getListOfGroups()
    
    model = sbml_document.getModel()
    groupsplugin = model.getPlugin("groups")
    group = groups.getGroup(group_id)
    
    reaction_set = set()
    species_set = set()
    
    for member in group.getListOfMembers():
        reaction_set.add(member.getIdRef())
    
    for reaction_id in reaction_set:
        reaction = model.getReaction(reaction_id)
        for reactant in reaction.getListOfReactants():
            species_set.add(reactant.getSpecies())
        for product in reaction.getListOfProducts():
            species_set.add(product.getSpecies())
    
    # remove
    for reaction in model.getListOfReactions():
        if not reaction.getId() in reaction_set:
            rmodel.removeReaction(reaction.getId())
    
    # also remove vertices
    for species in model.getListOfSpecies():
        if not species.getId() in species_set:
            rmodel.removeSpecies(species.getId())

    # and remove groups
    for group_index in range(len(rgroupslist)-1, -1, -1):
        group = rgroupslist.get(group_index)
        if not group.getId() == group_id:
            rgroupsplugin.removeGroup(group.getId())
    
    # remove fluxobjectives
    rflux = rmodel.getPlugin('fbc')
    rflux_list = rflux.getListOfObjectives()
    rflux_list.removeFromParentAndDelete()
    
    print rmodel.getListOfSpecies()
    print species_set
    print rmodel.getListOfReactions()
    print reaction_set
    print len(rgroupsplugin.getListOfGroups())

    rgroup = rgroupsplugin.getGroup(group_id)
    print rgroup
    
    #for member in rgroup.getListOfMembers():
    #    print member.getId()
    
    writer = SBMLWriter()
    writer.writeSBMLToFile(reduced_document, destination)
    return reduced_document
    
def sbml_to_json(sbml_document_name, destination):
    cobra_model_part = cobra.io.read_sbml_model(sbml_document_name)
    cobra.io.save_json_model(cobra_model_part, destination)
    

In [110]:
extract_sbml_by_group(document,'g4', 'g4.xml')
sbml_to_json('g4.xml', 'g4.json')

extract_sbml_by_group(document,'g6', 'g6.xml')
sbml_to_json('g6.xml', 'g6.json')

extract_sbml_by_group(document,'g12', 'g12.xml')
sbml_to_json('g12.xml', 'g12.json')

extract_sbml_by_group(document,'g14', 'g14.xml')
sbml_to_json('g14.xml', 'g14.json')


<ListOfSpecies[25]>
set(['M_pyr_c', 'M_pi_c', 'M_h2o2_c', 'M_o2_c', 'M_atp_c', 'M_coa_c', 'M_hco3_c', 'M_malcoa_c', 'M_h2o_c', 'M_h_c', 'M_ac_c', 'M_mal__L_c', 'M_adp_c', 'M_accoa_c', 'M_nad_c', 'M_amp_c', 'M_co2_c', 'M_3c3hmp_c', 'M_nadh_c', 'M_ppi_c', 'M_actp_c', 'M_lac__D_c', 'M_3mob_c', 'M_for_c', 'M_acald_c'])
<ListOfReactions[11]>
set(['R_ME1', 'R_POX2', 'R_ALDD2x', 'R_ACCOAC', 'R_PTAr', 'R_ACKr', 'R_ACS', 'R_IPPS', 'R_LDH_D', 'R_ACALD', 'R_PFL'])
1
<Group g4 "Pyruvate Metabolism">
<ListOfSpecies[60]>
set(['M_asn__L_c', 'M_acorn_c', 'M_hcys__L_c', 'M_akg_c', 'M_nh4_c', 'M_trdox_c', 'M_pi_c', 'M_asp__L_c', 'M_acser_c', 'M_sl26da_c', 'M_amp_c', 'M_o2_c', 'M_atp_c', 'M_5mta_c', 'M_pap_c', 'M_glu__L_c', 'M_thr__L_c', 'M_1pyr5c_c', 'M_trdrd_c', 'M_gln__L_c', 'M_aspsa_c', 'M_so3_c', 'M_nad_c', 'M_5mtr_c', 'M_met__L_c', 'M_uamr_c', 'M_amet_c', 'M_acglu_c', 'M_succ_c', 'M_nadp_c', 'M_h2o_c', 'M_h2s_c', 'M_so4_c', 'M_so4_e', 'M_gdp_c', 'M_ac_c', 'M_ala__L_c', 'M_ade_c', 'M_cys__L_c', 'M_a

### 2. For every reaction id in the model get the corresponding KEGG reaction id